In [2]:
import pandas as pd
import numpy as np
from google.oauth2 import service_account
from sklearn.linear_model import LogisticRegression
from datetime import datetime
import pandas_gbq

# fetch the data set and add IDs 
gamesDF = pd.read_csv("https://github.com/bgweber/Twitch/raw/master/Recommendations/games-expand.csv")
gamesDF['User_ID'] = gamesDF.index 
gamesDF['New_User'] = np.floor(np.random.randint(0, 10, gamesDF.shape[0])/9)

# train and test groups 
train = gamesDF[gamesDF['New_User'] == 0]
x_train = train.iloc[:,0:10]
y_train = train['label']
test = gamesDF[gamesDF['New_User'] == 1]
x_test = test.iloc[:,0:10]

# build a model
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict_proba(x_test)[:, 1]

# build a predictions data frame
resultDF = pd.DataFrame({'User_ID':test['User_ID'], 'Pred':y_pred} ) 
resultDF['time'] = str(datetime. now())

# save predictions to BigQuery 
table_id = "dsp_demo.user_scores"
project_id = "gameanalytics-199018"
credentials = service_account.Credentials.from_service_account_file('dsdemo.json')
pandas_gbq.to_gbq(resultDF, table_id, project_id=project_id, if_exists = 'replace', credentials=credentials)


/home/ec2-user/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
1it [00:03,  3.35s/it]


In [10]:
from google.oauth2 import service_account
import pandas_gbq

credentials = service_account.Credentials.from_service_account_file(
    'dsdemo.json',
)

project_id = "gameanalytics-199018"
sql = "SELECT * FROM  dsp_demo.user_scores"
df = pandas_gbq.read_gbq(sql, project_id=project_id, credentials=credentials)
df.head()


,User_ID,Pred,time
0,2735,0.112154,2019-11-04 01:06:02.929789
1,6597,0.209646,2019-11-04 01:06:02.929789
2,13159,0.209646,2019-11-04 01:06:02.929789
3,18179,0.209646,2019-11-04 01:06:02.929789
4,466,0.283951,2019-11-04 01:06:02.929789
